In [13]:
import MySQLdb
import cv2
import requests
import urllib.parse
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import datetime

import json


time1 = datetime.datetime.now()




connection = MySQLdb.connect(host = "127.0.0.1" , user = "root" , password = "zxcv2587", db = 'mltd' , charset = 'utf8') 
cursor = connection.cursor()



cursor.execute('SELECT `song_id` , `EX` FROM `mltd`.`maps`')
q = cursor.fetchall()


for song in q:

    url = song[1]
    req = requests.get(url)
    bs = BeautifulSoup(req.text , 'lxml')

    text = bs.select('#score_str')[0].text

#     text = json.loads(text)
    
#     print(text)
#     for i in text:
#         print(i , text[i])
    
    cursor.execute('''
    INSERT INTO `mltd`.`maps_info`(`song_id` , `content`) 
    VALUES (%s , %s)
    ''' ,
    (song[0] , text))
    
    








connection.commit()
cursor.close()
connection.close()
time2 = datetime.datetime.now()
print(time1)
print(time2)
print('papara')

2020-09-14 06:49:53.201102
2020-09-14 06:50:44.810054
papara


In [45]:
import MySQLdb
import cv2
import requests
import urllib.parse
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import datetime

import json


time1 = datetime.datetime.now()




connection = MySQLdb.connect(host = "127.0.0.1" , user = "root" , password = "zxcv2587", db = 'mltd' , charset = 'utf8') 
cursor = connection.cursor()



cursor.execute('SELECT `content` FROM `mltd`.`maps_info`')
q = cursor.fetchall()


for song in q:
    content = json.loads(song[0])
    con = content['Conductors']
#     print(con)
    notes = content['Notes']
    
    for note in notes:
        r = note['Ticks'] / 120
        if (r * 10) % 10 != 0:
            print(r)









connection.commit()
cursor.close()
connection.close()
time2 = datetime.datetime.now()
print(time1)
print(time2)
print('papara')

2020-09-15 08:22:29.522201
2020-09-15 08:22:30.174238
papara


In [67]:
import MySQLdb
import cv2
import requests
import urllib.parse
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import datetime

import math

import json

def get_streams(rhy):
    
    temp = []
    for i in rhy:
        if i not in temp:
            temp.append(i)
            
    
            
    for i in range(0 , len(temp) - 1):
        temp[i] = temp[i + 1] - temp[i]
    
    stream_max = 0
    streams = 0
    flag = 0
    count = 0
    
    for i in temp:
        
        if flag:
            
            if i == 12.0:
                count = count + 1
            else:
                if stream_max < count:
                    stream_max = count
                    
                count = 0
                flag = 0
        else:
            if i == 12.0:
                count = 2
                flag = 1
                streams = streams + 1
                if stream_max < count:
                    stream_max = count
                

                
    
    
    
    
    return [streams , stream_max]
    


time1 = datetime.datetime.now()




connection = MySQLdb.connect(host = "127.0.0.1" , user = "root" , password = "zxcv2587", db = 'mltd' , charset = 'utf8') 
cursor = connection.cursor()



cursor.execute('SELECT `song_id` , `content` FROM `mltd`.`maps_info`')
q = cursor.fetchall()







for song in q:
    print()
    
    move_total = 0
    flick = [0 , 0 , 0 , 0]
    speed_max = 1
    speed_min = 1
    circle_small = 0
    circle_big = 0
    circle_huge = 0
    
    sliders = 0
    content = json.loads(song[1])
    notes = content['Notes']
    line = 0
    ticks = []
    rhy = []
    rhy_left = []
    rhy_right = []
    
    streams = [0 , 0]
    
    pos = [0 , 0 , 0 , 0 , 0 , 0]
    
    for note in notes:
        
        
        move = abs(note['EndX'] - note['StartX'])
        move_total = move_total + move
        if note['FollowingNotes']:
            sliders = sliders + 1
            fn = note['FollowingNotes']
            en = fn[len(fn) - 1]

            if en['FlickDirection'] != 0:
                flick[0] = flick[0] + 1
                flick[en['FlickDirection']] = flick[en['FlickDirection']] + 1

            else:

                if en['Type'] == 100:
                    circle_huge = circle_huge + 1
                elif en['Size'] == 0:
                    circle_small = circle_small + 1
                elif en['Size'] == 1:
                    circle_big = circle_big + 1
                    
            if en['Ticks'] not in ticks:
                ticks.append(en['Ticks'])
            else:
                line = line + 1

            try:
                pos[en['EndX']] = pos[en['EndX']] + 1
            except:
                a = round(en['EndX'])
                pos[a] = pos[a] + 1

    
        if note['FlickDirection'] != 0:
            flick[0] = flick[0] + 1
            flick[note['FlickDirection']] = flick[note['FlickDirection']] + 1
            
        else:
            
            if note['Type'] == 100:
                circle_huge = circle_huge + 1
            elif note['Size'] == 0:
                circle_small = circle_small + 1
            elif note['Size'] == 1:
                circle_big = circle_big + 1
            
        if note['Speed'] > speed_max:
            speed_max = note['Speed']
        
        if note['Speed'] < speed_min:
            speed_min = note['Speed']
            
        if note['Ticks'] not in ticks:
            ticks.append(note['Ticks'])
        else:
            line = line + 1
        
        
        
        rhy_pos = note['Ticks'] / 120
        
        rhy.append(rhy_pos)
        if (note['EndX'] <= 2.5):
            rhy_left.append(rhy_pos)
        else:
            rhy_right.append(rhy_pos)
            
            
        pos[note['EndX']] = pos[note['EndX']] + 1
        
    temp = get_streams(rhy)
    streams = temp[0]
    stream_max = temp[1]
    
#     print(song[0])
#     print(move_total)
#     print(speed_max)
#     print(speed_min)
#     print(line)
#     print(circle_small)
#     print(circle_big)
#     print(circle_huge)
#     print(sliders)
#     print(flick)
#     print(pos)
#     print(streams)
#     print(stream_max)
    
    cursor.execute('''
    UPDATE `mltd`.`maps_info` 
    SET `move` = %s , `speed_max` = %s , `speed_min` =%s, 
    `flick_total` = %s , `flick_left` = %s , `flick_up` = %s , `flick_right` = %s ,
    `line` = %s , `sliders` = %s , 
    `circle_small` = %s , `circle_big` = %s , `circle_huge` = %s,
    `stream_total` = %s , `stream_max` = %s
    WHERE `song_id` = %s''' ,(move_total , speed_max , speed_min,
    flick[0] , flick[1] , flick[2] , flick[3],
    line , sliders , 
    circle_small , circle_big , circle_huge,
    streams , stream_max,
    song[0]))
    
    
#     print('rhy')
#     print(rhy_left)
#     break




#2 up
#1 3 left right

#12 stream
#24 single
#48 1pi


connection.commit()
cursor.close()
connection.close()
time2 = datetime.datetime.now()
print(time1)
print(time2)
print('papara')






















































































































2020-09-15 09:26:39.536409
2020-09-15 09:26:44.678703
papara
